In [1]:
!pip install langchain openai chromadb tiktoken tabulate sqlalchemy sqlalchemy-bigquery google-cloud-bigquery langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:

from google.colab import userdata
import os
import openai
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd "/content/drive/MyDrive/Colab Notebooks/시멘틱서치"

/content/drive/MyDrive/Colab Notebooks/시멘틱서치


In [5]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
service_account_file = "./first-project-394906-98976fac569b.json" # Change to where your service account key file is located
project = "first-project-394906"


In [8]:
dataset = "FACT_MSALES"
table = "MONTHLY_SALES"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'
db = SQLDatabase.from_uri(sqlalchemy_url)

## db chain

In [9]:
from langchain_experimental.sql import SQLDatabaseChain
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("24년 1월 OO업체, 5~10만원의 총 매출은 얼마야?")

## create_sql_agent 쿼리 생성

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "테이블의 행이 몇개야?"})
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


SELECT COUNT(*) AS total_rows
FROM AD


In [ ]:
db.run(response)

'[(42,)]'

## create_sql_agent

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType


agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


In [ ]:
agent_executor.run(
    "시기는 year_month입니다. 어느 시기에 모바일이 가장 많나요?"
)



> Entering new SQL Agent Executor chain...
 We need to find the time period with the highest number of mobile users.
Action: sql_db_query
Action Input: SELECT year_month, COUNT(*) AS num_users FROM mobile_users GROUP BY year_month ORDER BY num_users DESC LIMIT 1Error: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 404 Not found: Table first-project-394906:NAVER.mobile_users was not found in location US

Location: US
Job ID: 1e9259d5-1186-448f-8f0c-811cc7aa97e3

[SQL: SELECT year_month, COUNT(*) AS num_users FROM mobile_users GROUP BY year_month ORDER BY num_users DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/14/4xp6) The table name or location may be incorrect. We should use sql_db_schema to check the correct table name and location.
Action: sql_db_schema
Action Input: mobile_usersError: table_names {'mobile_users'} not found in database The table name may be incorrect. We should use sql_db_list_tables to check the available tables in the database.
Action:

'February 2024'

## 리트리버 툴킷
퓨샷 벡터 디비

In [11]:
!pip install tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.5 MB/s eta 0:00:00


In [19]:
few_shots = {
    "브랜드명 비바루비의 2댑스 정보를 알려줘": "SELECT CREMA_DEPTH2 FROM CAT_BRANDS WHERE BRAND_NAME = '비바루비';",
    "브랜드명 탑텐의 스타일 정보를 알려줘": "SELECT STYLE FROM CAT_BRANDS WHERE BRAND_NAME = '탑텐';",
    "상품아이디가 100 인 상품의 3댑스 정보를 알려줘": "SELECT DEPTH3 FROM CAT_NAMES WHERE PRODUCT_ID = 100;",
    "브랜드명 스파오의 주문월 23년 12월의 총 주문수를 알려줘" : "SELECT SUM(TOTAL_ORDERS) FROM DAILY_SALES WHERE BRAND_NAME = '스파오' AND ORDERED_YM = '2023-12';",
    "스파오의 주문일기준 주 시작일이 21년 12월 27일인 총 주문금액을 알려줘" : "SELECT SUM(TOTAL_SALES) FROM DAILY_SALES WHERE BRAND_NAME = '스파오' AND ORDERED_SOW = '2021-12-27';",
    "브랜드명 안국건강의 주문월 23년 9월 신제품 조회수를 알려줘" : "SELECT SUM(NPRODUCT_SALES) FROM MONTHLY_SALES WHERE BRAND_NAME = '안국건강' AND ORDERED_YM = '2023-09';",
    "안국건강의 주문월 23년 9월 신제품 상품수를 알려줘" : "SELECT SUM(NPRODUCT_COUNTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '안국건강' AND ORDERED_YM = '2023-09';",
    "체리코코의 주문일기준 주 종료일이 22년 1월 9일인 주간 방문자수를 확인해줘" : "SELECT SUM(WEEKLY_VISITORS) FROM WEEKLY_SALES WHERE BRAND_NAME = '체리코코' AND ORDERED_EOW = '2022-01-09';",
    "코오롱의 주문월 22년 1월 월 매출이 100~200만원인 유입자수를 알려줘" : "SELECT SUM(DEVICE_COUNTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '코오롱' AND ORDERED_YM = '2022-01' AND MONTHLY_PURCHASE = '100~200만원';",
    "코오롱의 주문월 23년 4월 구매율을 알려줘" : "SELECT AVG(ORDERS_PER_DEVICE) FROM MONTHLY_SALES WHERE BRAND_NAME = '코오롱' AND ORDERED_YM = '2023-04';",
    "뉴발란스의 전체 기간 전체 구매상품수를 확인해줘" : "SELECT SUM(OPRODUCT_COUNTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '뉴발란스';",
    "뉴발란스의 23년 리뷰 작성률을 알려줘" : "SELECT AVG(REVIEW_CREATE_RATE) FROM MONTHLY_SALES WHERE BRAND_NAME = '뉴발란스' AND SUBSTRING(ORDERED_YM,1,4) = '2023';",
    "젝시믹스의 22년 8월 유입당가치 알려줘" : "SELECT AVG(VALUE_PER_DEVICE) FROM MONTHLY_SALES WHERE BRAND_NAME = '젝시믹스' AND ORDERED_YM = '2022-08';",
    "젝시믹스의 24년 1월 신규구매자의 구매단가 알려줘" : "SELECT AVG(PRICE_CENTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '젝시믹스' AND ORDERED_YM = '2024-01' AND ORDER_TYPE = 'NEW';",
    "공구홀릭의 대카테고리가 상의이고 주문일기준 시작일이 2023년 7월 9일의 총 매출을 확인해줘" : "SELECT SUM(TOTAL_SALES) FROM WEEKLY_SALES WHERE BRAND_NAME = '공구홀릭' AND DEPTH2 = '상의' AND ORDERED_SOW = '2023-07-09';",
    "45번가의 24년 2월 신상품 중 가격대 1만원대 총 매출액을 알려줘" : "SELECT SUM(TOTAL_SALES) FROM MONTHLY_SALES WHERE BRAND_NAME = '45번가' AND ORDERED_YM = '2024-02' AND PRODUCT_TYPE = 'NEW' AND PRICE_BAND = '1만원';",
    "황씨네옷장 23년 7월 키워드 점수 확인해줘" : "SELECT SUM(SENTIMENT) FROM MONTHLY_SALES WHERE BRAND_NAME = '황씨네옷장' AND ORDERED_YM = '2023-07';",
    "황씨네옷장 23년 7월 긍정키워드 수 알려줘" : "SELECT COUNT(PKEYWORD_COUNTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '황씨네옷장' AND ORDERED_YM = '2023-07';",
    "아모레퍼시픽 23년 부정 인지율" : "SELECT AVG(NSENTIMENT_RATE) FROM MONTHLY_SALES WHERE BRAND_NAME = '아모레퍼시픽' AND SUBSTRING(ORDERED_YM,1,4) = '2023';",
    "아모레퍼시픽 23년 1월 2위~3위 상품의 총 리뷰수 확인해줘" : "SELECT SUM(REVIEW_COUNTS) FROM MONTHLY_SALES WHERE BRAND_NAME = '아모레퍼시픽' AND ORDERED_YM = '2023-01' AND RANK_BAND = '2위~3위';",
    "네이버 CLASS가 age인 리스트": "SELECT * FROM AD WHERE CLASS = 'age';",
    "네이버 모바일 전체 검색량": "SELECT SUM(MOBILE) FROM AD;",
}

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

In [21]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
이 도구는 유사한 예시를 이해하여 사용자 질문에 적용하는 데 도움이 됩니다.
이 도구에 입력하는 내용은 사용자 질문이어야 합니다.
"""

retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]

In [22]:
dataset = "FACT_RANK"
table = "MONTHLY_SALES"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'
db = SQLDatabase.from_uri(sqlalchemy_url)

In [23]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
먼저 알고 있는 비슷한 예제를 가져와야 합니다.
예제가 쿼리를 구성하기에 충분하다면 쿼리를 작성할 수 있습니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

In [ ]:
agent.run("브랜드명 OO업체 23년 4월 1위 상품의 총 판매량")

## 리트리버 툴킷
테이블 설명 + 고유 컬럼 매칭 or  
테이블 value 상세 정보

In [ ]:
print(db.table_info)

In [ ]:
import ast
import re


# def run_query_save_results(db, query):
#     res = db.run(query)
#     res = [el for sub in ast.literal_eval(res) for el in sub if el]
#     res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
#     return res


# artists = run_query_save_results(db, "SELECT name FROM artists")
# albums = run_query_save_results(db, "SELECT title FROM albums")
kor
eng


In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

texts = kor + eng

embeddings = OpenAIEmbeddings()
vector_db = FAISS.from_texts(texts, embeddings)
retriever = vector_db.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="col_names",
    description="테이블 스키마의 컬럼 이름이 실제로 어떻게 쓰여졌는지 알아내는 데 사용합니다.",
)

custom_tool_list = [retriever_tool]

In [ ]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
사용자가 고유명사를 기준으로 필터링해 달라고 요청하는 경우, 먼저 col_names 도구를 사용하여 철자를 확인해야 합니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
)

In [ ]:
agent.run("brand_name이 나이키이고 시작주가 2022년 1월 1일인 테이블 행은 몇 개 인가요?")